In [2]:
import numpy as np    
import pandas as pd    
import matplotlib.pyplot as plt  
import os  
import cartopy.crs as ccrs  
import cartopy.feature as cfeature  
import matplotlib.ticker as mticker  
from matplotlib.ticker import MultipleLocator, FuncFormatter

In [16]:
df_0 = pd.read_excel(r'E:\all_draw\CTD\营养盐数据.xlsx')
df_0['站位'] = df_0['站位'].fillna(method='ffill')
df_0 = df_0.rename(columns={'站位':'站位名'})
df_0

,站位名,深度(m),经度(E),纬度(N),磷酸盐,硝酸盐,亚硝酸盐,铵盐,硅酸盐
0,NaN,NaN,NaN,NaN,μmol/L,μmol/L,μmol/L,μmol/L,μmol/L
1,QB03,5.0,142.076788,0.002646,0.177419,0.285714,0.142857,0.428571,2.921429
2,QB03,25.0,142.076788,0.002646,0.16129,0.214286,0.142857,0.428571,1.553571
3,QB03,50.0,142.076788,0.002646,0.177419,0.285714,0.142857,0.5,1.128571
4,QB03,75.0,142.076788,0.002646,0.209677,0.428571,0.285714,0.5,1.564286
...,...,...,...,...,...,...,...,...,...
556,QB17,1000.0,135.092234,17.931348,2.677419,39.571429,0.285714,4.214286,58.542857
557,QB17,1500.0,135.092234,17.931348,2.903226,38.142857,0.285714,4.5,76.375
558,QB17,2000.0,135.092234,17.931348,2.629032,39,0.357143,5.071429,158.178571
559,QB17,3000.0,135.092234,17.931348,2.854839,36.857143,0.571429,8.5,158.628571


In [4]:
stations = []
for i in df_0['站位']:
    stations.append(i)
stations = list(set(stations))
print(stations)

['QB19', 'FB01', 'QB05', 'N3-03', 'QB17', 'E163-01', 'QB03', 'E163-05', 'Mk-03', 'QB11', 'E163-09', 'E163-04', 'QB10', 'E142-06', 'QB12', 'QB08', 'N3-05', 'FB02', nan, 'QB07', 'E163-06', 'E163-08', 'E142-07', 'E163-02', 'QB09', 'E163-11', 'QB04', 'QB02', 'QB16', 'QB18', 'E142-5', 'N3-01', 'E142-10', 'E163-10', 'QB-07new', 'QB06', 'E163-03', 'E163-07', 'EQ-03', 'QB14', 'QB01', 'QB15']


In [17]:
import re

def dms_to_decimal(dms):
    # 使用正则表达式匹配度分秒格式，包括不同的分隔符和方向指示符
    match = re.match(r"(\d+)[°\s](\d+)[′'’\s](\d+(\.\d+)?)[\"\s]?(N|S|E|W)?", dms)
    if match:
        degrees = float(match.group(1))
        minutes = float(match.group(2))
        seconds = float(match.group(3))
        direction = match.group(5)
        decimal = degrees + (minutes / 60) + (seconds / 3600)
        if direction in ['S', 'W']:
            decimal = -decimal
        return decimal
    else:
        raise ValueError(f"无法解析的度分秒格式: {dms}")

# 示例数据
dms_str = "130°00′12\"E"
print(dms_to_decimal(dms_str))


130.00333333333333


In [5]:
def dms_to_decimal(dms):
    """将度分秒转换为十进制度数"""
    dms = dms.replace('°', ' ').replace('′', ' ').replace('″', ' ').replace('"E', ' ').replace('"N', ' ').replace('"S', ' ')
    parts = dms.split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal

In [18]:
stations_ctd = os.listdir(r'E:\all_draw\CTD\CTD')
lonlat = []  
for i in stations_ctd:
    if i.endswith('.xls'):
        df = pd.read_excel(r'E:\all_draw\CTD\CTD\%s' % i, skiprows=1, header=1)
        df = df.drop(0)
        longitude = df['经度'].apply(dms_to_decimal).values # type: ignore
        latitude = df['纬度'].apply(dms_to_decimal).values  # type: ignore
        name = df['站位名'].values
        lonlat.append((longitude[3], latitude[3], name[3]))
lonlat = pd.DataFrame(lonlat, columns=['lon', 'lat', '站位名'])
data_path = pd.merge( lonlat, df_0, on='站位名')

In [19]:
# 自定义刻度格式化函数
def lon_formatter(x, pos):
    if x < 0:
        return f'{abs(int(x))}°W'
    elif x > 180:
        return f'{360-int(x)}°W'
    else:
        return f'{int(x)}°E'

def lat_formatter(x, pos):
    if x > 0:
        return f'{int(x)}°N' 
    else:
        return f'{int(x)}°S'

In [66]:

#####################################绘制scatter###############################################
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.rcParams['font.size'] = 24

stations_path = []
for i in data_path['站位名']:
    stations_path.append(i)
stations_path = list(set(stations))

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax.set_extent([135, 165.001, -5, 20.001], crs=ccrs.PlateCarree())

for station_path in stations_path:
    data = data_path[data_path['站位名'] == station_path]
    max_depth = data['深度(m)'].max()
    # data_1 = data_path[(data_path['深度(m)'] > 3000) & (data_path['深度(m)'] <= 4000)]
    data_1 = data_path[data_path['深度(m)'] == max_depth]##########################################################################################
    station = data_1['站位名']
    lon = data_1['lon'].values
    lat = data_1['lat'].values
    sio4 = data_1['硅酸盐'].values
    sc = ax.scatter(lon, lat, s=100, c=sio4, cmap='Reds', marker='o', alpha=1, edgecolors='k', transform=ccrs.PlateCarree(), zorder=10, linewidths=0.5, vmin=140, vmax=150)

    
ax.spines[:].set_linewidth(2)
ax.add_feature(cfeature.BORDERS, linewidth=2)
ax.add_feature(cfeature.COASTLINE, linewidth=2)
ax.add_feature(cfeature.RIVERS, linewidth=2)
ax.add_feature(cfeature.LAND, edgecolor='black', zorder=9)
#####################################################
import xarray as xr
ds = xr.open_dataset(r"E:\all_draw\data\Copernicus_bathy\cmems_mod_glo_phy_my_0.083deg_static_1720691864705.nc")
elevation = ds['deptho'].values
longitude = ds['longitude'].values
latitude = ds['latitude'].values
mask = elevation > 4000 ##################################################################################################
mask_1 = elevation > 2000
mask_2 = elevation > 3000
zhe_gai = ax.contourf(longitude, latitude, mask, levels=[0, 0.5], colors='grey', transform=ccrs.PlateCarree(), alpha = 0.3)
zhe_gai_1 = ax.contourf(longitude, latitude, mask_1, levels=[0, 0.5], colors='orange', transform=ccrs.PlateCarree(), alpha = 0.7)
zhe_gai_2 = ax.contourf(longitude, latitude, mask_2, levels=[0, 0.5], colors='grey', transform=ccrs.PlateCarree(), alpha = 0.7)
######################################################
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
gl = ax.gridlines(draw_labels=False, linewidth=1, color='white', alpha=0.15, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15, 'color': 'black'}
gl.ylabel_style = {'size': 15, 'color': 'black'}
ax.set_xticks(np.arange(135, 165.001, 10))
ax.set_yticks(np.arange(0, 20.001, 5))
ax.xaxis.set_major_formatter(FuncFormatter(lon_formatter))
ax.yaxis.set_major_formatter(FuncFormatter(lat_formatter))
    
cbar = plt.colorbar(sc, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.65)
cbar.set_label('SiO4 (μmol/L)', fontsize=18)
    
# plt.show()
plt.savefig('2021cruice_scatter_maxdepth.png', bbox_inches='tight', dpi=300)
plt.close()